In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
folder_list = ["input_junio2019_si_kine_mod", "input_junio2019_no_kine_mod"]
subfolder_list = ["RegularTimeWindow", "DoubleTimeWindow", "NoTimeWindow"]

In [3]:
patient_file_prefix = "output_patients"
attwindow_file_prefix = "output_attwindows"
tptwindow_file_prefix = "output_tptwindows"
vehicle_file_prefix = "output_vehicles"

In [4]:
def window_deviation(t, start, end):
    return max(start-t, 0, t-end)

In [11]:
def stats_simulation(results_folder):
    
    # lectura de archivos
    
    for file in os.listdir(results_folder):
        filepath = f"{results_folder}/{file}"

        if file.startswith(patient_file_prefix):
            df_patients = pd.read_excel(filepath, usecols=lambda x: 'Unnamed' not in x)

        elif file.startswith(vehicle_file_prefix):
            df_vehicles = pd.read_excel(filepath, usecols=lambda x: 'Unnamed' not in x)

        elif file.startswith(attwindow_file_prefix):
            df_attwindows = pd.read_excel(filepath)
            df_attwindows.columns = ["id","start","end"]
            df_attwindows.set_index("id", inplace=True)

        elif file.startswith(tptwindow_file_prefix):
            df_tptwindows = pd.read_excel(filepath)
            df_tptwindows.columns = ["id","start","end"]
            df_tptwindows.set_index("id", inplace=True)

    # estadisticas pacientes con transporte
    
    duration_to_center = []
    duration_pre_attention = []
    duration_post_attention = []
    duration_from_center = []
    
    for index, df in df_patients[df_patients.transport == True].groupby(["date"]):
        duration_to_center.append((df.arrival_time - df.pickup_time).mean())
        duration_pre_attention.append((df.attention_time - df.arrival_time).mean())
        duration_post_attention.append((df.departure_time - df.release_time).mean())
        duration_from_center.append((df.dropoff_time - df.departure_time).mean())
    
    mean_to_center = np.mean(duration_to_center)
    mean_pre_attention = np.mean(duration_pre_attention)
    mean_post_attention = np.mean(duration_post_attention)
    mean_from_center = np.mean(duration_from_center)
    mean_total = mean_to_center + mean_pre_attention + mean_post_attention + mean_from_center
    
    print("(0) Tiempo en el sistema:")
    print(f"--- Traslado al centro: {mean_to_center:.0f} min")
    print(f"--- Espera pre-atencion: {mean_pre_attention:.0f} min")
    print(f"--- Espera post-atencion: {mean_post_attention:.0f} min")
    print(f"--- Traslado desde centro: {mean_from_center:.0f} min")
    print(f"--- Total (sin atencion): {mean_total:.0f} min")
    
    patients_std = []
    patients_freq = []
    patients_dev = []
    
    for index, df in df_patients[df_patients.transport == True].groupby(["id"]):
        if len(df) > 1:
            patients_std.append(df.pickup_time.std())

            window_start = df_tptwindows.loc[index, "start"]
            window_end = df_tptwindows.loc[index, "end"]
            att_deviation = df.pickup_time.apply(lambda x: window_deviation(x, window_start, window_end))
            n_total = len(att_deviation)
            n_positive = (att_deviation > 0.5).sum()

            if n_positive > 0:
                patients_freq.append(n_positive / n_total)
                patients_dev.append(att_deviation.sum() / n_positive)

    print("(1) Pacientes con transporte:")
    print(f"--- Dispersion media de hora recogida: {np.mean(patients_std):.0f} min")
    print(f"--- Cantidad pacientes con alguna desviacion: {len(patients_dev)} de {len(patients_std)} ({len(patients_dev)/len(patients_std):.2f})")
    print(f"--- Frecuencia media de desviacion en hora recogida: {np.mean(patients_freq):.2f}")
    print(f"--- Desviacion media de hora recogida: {np.mean(patients_dev):.0f} min")

    # estadisticas pacientes sin transporte
    
    patients_std = []
    patients_freq = []
    patients_dev = []
    
    for index, df in df_patients[(df_patients.transport == False) & (df_patients.repose == True)].groupby(["id"]):
        if len(df) > 1:
            patients_std.append(df.attention_time.std())

            window_start = df_attwindows.loc[index, "start"]
            window_end = df_attwindows.loc[index, "end"]
            att_deviation = df.attention_time.apply(lambda x: window_deviation(x, window_start, window_end))
            n_total = len(att_deviation)
            n_positive = (att_deviation > 0.5).sum()

            if n_positive > 0:
                patients_freq.append(n_positive / n_total)
                patients_dev.append(att_deviation.sum() / n_positive)

    print("(2) Pacientes sin transporte:")
    print(f"--- Dispersion media de hora atencion: {np.mean(patients_std):.0f} min")
    print(f"--- Cantidad pacientes con alguna desviacion: {len(patients_dev)} de {len(patients_std)} ({len(patients_dev)/len(patients_std):.2f})")
    print(f"--- Frecuencia media de desviacion en hora atencion: {np.mean(patients_freq):.2f}")
    print(f"--- Desviacion media de hora atencion: {np.mean(patients_dev):.0f} min")
    
    # estadisticas vehiculos

    df_fleet = df_vehicles.loc[~df_vehicles.id.str.contains("Taxi")]
    fleet_trips = df_fleet.n_trips.sum()
    fleet_routes = df_fleet.n_routes.sum()
    fleet_time = df_fleet.busy_time.sum()
    
    df_taxis = df_vehicles.loc[df_vehicles.id.str.contains("Taxi")]
    taxis_trips = df_taxis.n_trips.sum()
    taxis_routes = df_taxis.n_routes.sum()
    taxis_time = df_taxis.busy_time.sum()

    print("(3) Utilizacion vehiculos:")
    
    print("Vehiculos propios:")
    print(f"--- Traslados: {fleet_trips}")
    print(f"--- Viajes: {fleet_routes}")
    print(f"--- Ocupacion: {fleet_trips / fleet_routes:.1f}")
    print(f"--- Minutos: {fleet_time}")
    
    print("Taxis:")
    print(f"--- Traslados: {taxis_trips}")
    print(f"--- Viajes: {taxis_routes}")
    print(f"--- Ocupacion: {taxis_trips / taxis_routes:.1f}")
    print(f"--- Minutos: {taxis_time}")
    
    print("Total:")
    print(f"--- Traslados: {fleet_trips + taxis_trips}")
    print(f"--- Viajes: {fleet_routes + taxis_routes}")
    print(f"--- Ocupacion: {(fleet_trips + taxis_trips) / (fleet_routes + taxis_routes):.1f}")
    print(f"--- Minutos: {fleet_time + taxis_time}")


In [12]:
for subfolder in subfolder_list:
    for folder in folder_list:
        results_folder = f"{folder}/{subfolder}"
        print(results_folder)
        stats_simulation(results_folder)
        print()

input_junio2019_si_kine_mod/RegularTimeWindow
(0) Tiempo en el sistema:
--- Traslado al centro: 44 min
--- Espera pre-atencion: 48 min
--- Espera post-atencion: 59 min
--- Traslado desde centro: 32 min
--- Total (sin atencion): 184 min
(1) Pacientes con transporte:
--- Dispersion media de hora recogida: 34 min
--- Cantidad pacientes con alguna desviacion: 20 de 65 (0.31)
--- Frecuencia media de desviacion en hora recogida: 0.35
--- Desviacion media de hora recogida: 72 min
(2) Pacientes sin transporte:
--- Dispersion media de hora atencion: 29 min
--- Cantidad pacientes con alguna desviacion: 43 de 92 (0.47)
--- Frecuencia media de desviacion en hora atencion: 0.41
--- Desviacion media de hora atencion: 62 min
(3) Utilizacion vehiculos:
Vehiculos propios:
--- Traslados: 1021
--- Viajes: 250
--- Ocupacion: 4.1
--- Minutos: 18727
Taxis:
--- Traslados: 271
--- Viajes: 87
--- Ocupacion: 3.1
--- Minutos: 5803
Total:
--- Traslados: 1292
--- Viajes: 337
--- Ocupacion: 3.8
--- Minutos: 24530



C:\Users\Felipe\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\Felipe\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


(3) Utilizacion vehiculos:
Vehiculos propios:
--- Traslados: 1063
--- Viajes: 234
--- Ocupacion: 4.5
--- Minutos: 17655
Taxis:
--- Traslados: 229
--- Viajes: 71
--- Ocupacion: 3.2
--- Minutos: 4808
Total:
--- Traslados: 1292
--- Viajes: 305
--- Ocupacion: 4.2
--- Minutos: 22463

input_junio2019_no_kine_mod/NoTimeWindow
(0) Tiempo en el sistema:
--- Traslado al centro: 39 min
--- Espera pre-atencion: 42 min
--- Espera post-atencion: 57 min
--- Traslado desde centro: 30 min
--- Total (sin atencion): 168 min
(1) Pacientes con transporte:
--- Dispersion media de hora recogida: 92 min
--- Cantidad pacientes con alguna desviacion: 0 de 65 (0.00)
--- Frecuencia media de desviacion en hora recogida: nan
--- Desviacion media de hora recogida: nan min
(2) Pacientes sin transporte:
--- Dispersion media de hora atencion: 110 min
--- Cantidad pacientes con alguna desviacion: 0 de 92 (0.00)
--- Frecuencia media de desviacion en hora atencion: nan
--- Desviacion media de hora atencion: nan min
(3) Ut